In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import datetime, date, timedelta

# statistical analysis
import statsmodels.api as sm
from statsmodels import regression
import scipy.stats

import yfinance as yfin
yfin.pdr_override()
from var import VaR

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
sns.set_theme(rc={'axes.formatter.limits': (-8, 9)})
style.use('seaborn-whitegrid')
plt.rcParams["figure.figsize"] = (10,6)

# 0. Weekly Tasks
Prepare a presentation using Panopto (PowerPoint slides with Video Narration is an acceptable alternative) addressing the following points:
What scenario were you assigned as the result of the random number exercise?

- What were your considerations in determining an appropriate investment strategy for that scenario?

- What asset allocation did you implement that you felt best supported that investment strategy?

- What assets did you invest in consistent with that asset allocation and strategy? (Assets, Price at time of investment, % of total investment)

- Present Value, Profit/Loss to date, VaR, SHARPE, VWAP, Alpha (Other metrics that you feel important in support of your strategy) for your portfolio 

What reallocation of assets would you recommend at this time (if any) and why?
Post your presentation accessible to your colleagues for comment and comment on the presentations by your colleagues.

# 1. Variables

In [ ]:
start_date = datetime(2022, 1, 1)
purchase_date = datetime(2023, 9, 11)
purchase_date_str = '2023-9-11'
end_date = datetime.now()
end_date_str = end_date.strftime("%Y-%m-%d")

alpha = 0.05
period = '5y'

# 2. Functions

# 3. Load Data

## 3.1 Load Variables

In [ ]:
var_lst = ['^TNX']
data = pdr.get_data_yahoo(var_lst, start=start_date, end=end_date )['Close']
rf_rate = round((data.mean()),4)
rf_rate

In [ ]:
var_lst = ['SPY']
bench_df = pdr.get_data_yahoo(var_lst, start=start_date, end=end_date)['Close']
bench_econ_df = pd.DataFrame(bench_df)
bench_econ_df.columns = ['SPY']

benchPercent_econ_df = bench_econ_df.pct_change()[1:]
benchPercent_econ_df.reset_index(inplace=True)
benchPercent_econ_df.head()

In [ ]:
var_lst = ['FIBUX']
bench_df = pdr.get_data_yahoo(var_lst, start=start_date, end=end_date)['Close']
bench_int_df = pd.DataFrame(bench_df)
bench_int_df.columns = ['FIBUX']

benchPercent_int_df = bench_int_df.pct_change()[1:]
benchPercent_int_df.reset_index(inplace=True)
benchPercent_int_df.head()

## 3.2 Load Portfolio

In [ ]:
folio_df = pd.read_csv('https://raw.githubusercontent.com/dsimband/DATA618/main/w7/data/DATA618_Portfolio.csv', 
                 dtype={
                    'ID': 'int',
                    'Price': 'float',
                    'Shares': 'float',
                    'Value': 'float', 
                 })

folio_df = folio_df[folio_df['Shares'] > 0]
folio_df = folio_df.groupby(['Ticker','BondName','Class','Sub_Class'])[['Shares','Value']].sum()
folio_df.reset_index(inplace=True)

# calculate portfilio percentage
portfolio_total = folio_df['Value'].sum()
folio_df['port_percent'] = folio_df['Value'] / portfolio_total

# Class Portfolios
folio_econ_df = folio_df[folio_df['Class'] == 'Economically Sensitive']
folio_int_df = folio_df[folio_df['Class'] == 'Interest Rate Sensitive']

folio_df

In [ ]:
folio_df.groupby(['Class'])[['Value']].sum()

In [ ]:
initial_investment = folio_df['Value'].sum() * 1000
initial_investment

# 4. Portfolio

## 4.1 Load Pricing Time Series

In [ ]:
# ticker symbols
ticker_lst = list(folio_df['Ticker'])
print('ticker #:', len(ticker_lst))

# portfolio weights
weight_lst = (folio_df['port_percent'].values)
print('price #:', len(weight_lst))

#Download closing prices
price_df = pdr.get_data_yahoo(ticker_lst, start=start_date, end=end_date)['Close']
price_df['C_A_S_H'] = 1
print('price_df #:', price_df.shape)

#From the closing prices, calculate periodic returns
return_df = price_df.pct_change()
return_df.fillna(0, inplace=True)
return_df.index = pd.to_datetime(return_df.index)
print('return_df #:', len(return_df.columns))

## 4.2 Portfolio Returns

In [ ]:
port_ret_weighted= return_df.mul(weight_lst, axis=1)
print('port_ret_weighted #:', port_ret_weighted.shape)

return_df['Portfolio'] = port_ret_weighted.sum(axis=1)
print('return_df #:', return_df.shape)

In [ ]:
weight_lst

# 5. Calculate Portfolio Value

In [ ]:
shares_df = folio_df[['Ticker','Shares']]
shares_df.set_index('Ticker', drop=True, inplace=True)
shares_df.head()

In [ ]:
m_df = price_df.copy()
m_df.reset_index(inplace=True)
m_df = m_df.melt(id_vars=['Date'])
m_df.columns = ['Date','Ticker','Price']
m_df.head()

In [ ]:
l_df = folio_df[['Ticker','Class','Sub_Class']] #.drop_duplicates() 
print('l_df: ',l_df.shape)

In [ ]:
merge_df = m_df.merge(l_df, how='outer', left_on='Ticker', right_on='Ticker').merge(shares_df,how='outer' ,left_on='Ticker', right_on='Ticker')
merge_df['share_value'] = merge_df['Price'] * merge_df['Shares']
merge_df['perc_share_value'] = merge_df.groupby(['Ticker'])[['share_value']].pct_change()

print('merge_df: ',merge_df.shape)

In [ ]:
merge_df

In [ ]:
folioValue_df = merge_df.groupby(['Date'])[['share_value']].sum().reset_index()
folioValue_df['pct_change'] = folioValue_df['share_value'].pct_change()

print('folioValue_df: ',folioValue_df.shape)

In [ ]:
fv_df = merge_df.groupby(['Date','Class'])[['share_value']].sum().reset_index()
fv_df['pct_change'] = fv_df.groupby(['Class'])[['share_value']].pct_change()

print('folioValue_df: ',fv_df.shape)

## 5.1 Portfolio Value

In [ ]:
folio_df

In [ ]:
folio_df.groupby(['Class','Sub_Class'])[['Value']].sum()

In [ ]:
folio_df[['Value']].sum()

In [ ]:
round(merge_df[merge_df['Date'] == purchase_date_str].groupby(['Class','Sub_Class'])[['share_value']].sum(),0)

In [ ]:
round(merge_df[merge_df['Date'] == purchase_date_str][['share_value']].sum(),0)

In [ ]:
folioValue_df[folioValue_df['Date'] == purchase_date_str]

In [ ]:
round(merge_df[merge_df['Date'] == end_date_str].groupby(['Class','Sub_Class'])[['share_value']].sum(),0)

In [ ]:
round(merge_df[merge_df['Date'] == end_date_str][['share_value']].sum(),0)

In [ ]:
folioValue_df[folioValue_df['Date'] == end_date_str]

# 6. Correlation

In [ ]:
m_df = merge_df.groupby(['Date','Class'])[['share_value']].sum()
m_df['pct_change'] = m_df.groupby(['Class'])[['share_value']].pct_change()
m_df.reset_index(inplace=True)
m_df = m_df.pivot(index=['Date'], columns=['Class'])['pct_change']
m_df

In [ ]:
p_df = m_df.merge(benchPercent_econ_df, how='outer', left_on='Date', right_on='Date') \
            .merge(benchPercent_int_df,how='outer' ,left_on='Date', right_on='Date')
p_df.drop(labels=['Date','Cash'], inplace=True, axis=1)
p_df.columns = ['Economic','Interest','SPY','FHNFX']
p_df

In [ ]:
plt.figure(figsize=(10,5))
mask = np.zeros_like(p_df.corr())
mask[np.triu_indices_from(mask)] = True
sns.set_style("white")
_p = sns.heatmap(p_df.corr().round(2), 
                 annot=True, mask=mask, 
                 cmap="plasma", annot_kws={"size": 10})

# 7. Value at Risk

# 8. Calculate VaR

In [ ]:
r_df = return_df.drop(labels='Portfolio', axis=1)[1:]
print('r_df', r_df.shape)

In [ ]:
l2 = list(folio_df['Ticker'])
l1 = r_df.columns
list_dif = set(l2).symmetric_difference(set(l1))

print('l1: ', len(l1), 'l2: ', len(l2), 'dif: ', list_dif)

In [ ]:
var = VaR(r_df, weight_lst, alpha=[0.05,0.025,0.01])
var

In [ ]:
var.info

In [ ]:
var.historic()

In [ ]:
print('VaR(95.0):', var.historic()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.historic()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
var.parametric()

In [ ]:
print('VaR(95.0):', var.parametric()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.parametric()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
var.monte_carlo()

In [ ]:
print('VaR(95.0):', var.monte_carlo()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.monte_carlo()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
l2 = list(folio_df['Ticker'])
l1 = r_df.columns
list_dif = set(l2).symmetric_difference(set(l1))


print('l1: ', len(l1), 'l2: ', len(l2), 'dif: ', list_dif)

In [ ]:
var = VaR(r_df, weight_lst, alpha=[0.05,0.025,0.01])
#var = VaR(r_df, weight_lst, alpha=0.05)
var

In [ ]:
var.info

In [ ]:
var.historic()

In [ ]:
print('VaR(95.0):', var.historic()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.historic()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
var.parametric()

In [ ]:
print('VaR(95.0):', var.parametric()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.parametric()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
var.monte_carlo()

In [ ]:
print('VaR(95.0):', var.monte_carlo()['VaR(95.0)'][0]*100)
print('Portfolio VaR(95.0)', var.monte_carlo()['VaR(95.0)'][0]*portfolio_total*1000)

In [ ]:
var.monte_carlo(stressed=True)

# Backtest

In [ ]:
bth = var.backtest(method='h')

In [ ]:
var.evaluate(bth)

In [ ]:
var.var_plot(bth)

In [ ]:
var.cvar_plot(bth)

In [ ]:
var.cdar_plot(bth)